In [1]:
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, FunctionMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain import hub
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Your further code goes here


In [2]:
import re
import time
import pandas as pd
from pprint import pprint

from gsheet_handler import GSheetHandler
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')


no docx
no docx


In [3]:
# main.py
import json
import os
import telebot

# Загрузка конфигурации из JSON-файла
with open('config.json') as config_file:
    config = json.load(config_file)

# Установка переменных окружения
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="nlmk_bot"

In [4]:
from datetime import datetime
from langchain import hub

def evaluate_news_article(text):
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    prompt = hub.pull("mlenparrot/nlmk_aiparse")
    params = {
        'text': text
    }
    chain = prompt | llm
    result = chain.invoke(params)
    return result

#result = evaluate_news_article(text)



#print(result)

In [5]:
parse_raw = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=1997913753#gid=1997913753', 
    2)[['url', 'content']][:3]
parse_raw

,url,content
0,https://edtechmagazine.com/,Today's date: 2024-06-14\nK12 and Higher Educa...
1,https://social.evraz.com/news/,Today's date: 2024-06-14\nНовости / ЕВРАЗ - Со...
2,https://severstal.com/rus/media/,


In [6]:
def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list
nested_list = [1, [2, 3], [4, [5, 6]], 7, [8, [9, [10]]]]
flat_list = flatten_list(nested_list)
print(flat_list)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [7]:
from tqdm import tqdm
parse_raw['combined_text'] = parse_raw['content'] + " " + parse_raw['url'] 
results = []
for combined_text in tqdm(parse_raw['combined_text'], desc="Processing articles"):
    evaluation_result = evaluate_news_article(combined_text).get('news_list', [])
    results.append(evaluation_result)

Processing articles: 100%|██████████| 3/3 [00:05<00:00,  1.96s/it]


In [8]:
news_df = pd.DataFrame(flatten_list(results))
news_df

,link,news_date,news_text,news_topic
0,https://edtechmagazine.com/,,,
1,https://social.evraz.com/news/news/itogi-grant...,2024-06-14,,Итоги грантового конкурса социальных проектов ...
2,https://severstal.com/rus/media/,,,


In [9]:
ggl.update_gsheet_with_df(news_df,
                          'https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit#gid=0', 
                        3,
                        
                         )#  integer_columns=[3],
                        #  float_columns=[4,5])

In [10]:
news_df

,link,news_date,news_text,news_topic
0,https://edtechmagazine.com/,,,
1,https://social.evraz.com/news/news/itogi-grant...,2024-06-14,,Итоги грантового конкурса социальных проектов ...
2,https://severstal.com/rus/media/,,,


In [11]:
rss_raw = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=1997913753#gid=1997913753', 
    1)[['title', 'link', 'source', 'content', 'formatted_dates']]
rss_raw

,title,link,source,content,formatted_dates
0,Have Good Health Insurance? Too Bad. You Could...,https://www.forbes.com/sites/reneehsia/2024/06...,Forbes - Innovation,Health insurance doesn't protect you from rece...,2024-06-14
1,Conor McGregor Is Out And UFC 303 Main Card Ge...,https://www.forbes.com/sites/brianmazique/2024...,Forbes - Innovation,Conor McGregor is officially off the UFC 303 c...,2024-06-14
2,The AI Revolution Comes for Farmers Growing a ...,https://www.bloomberg.com/news/articles/2024-0...,Bloomberg Technology,"<p>In its first test in Malawi, an AI app is h...",2024-06-14
3,AI Demand Helps Korean Chip Export Prices to R...,https://www.bloomberg.com/news/articles/2024-0...,Bloomberg Technology,The price of South Korea’s semiconductor expor...,2024-06-14
4,Microsoft Pulls Back Wide Release of Criticize...,https://www.bloomberg.com/news/articles/2024-0...,Bloomberg Technology,"Microsoft Corp., facing scrutiny over the secu...",2024-06-14
...,...,...,...,...,...
68,Силы ПВО отразили массовую атаку беспилотников...,https://www.interfax.ru/russia/966536,Интерфакс,Массовая атака дронов отражена в Ростовской об...,2024-06-14
69,Кровля горящего здания обрушилась на севере Мо...,https://www.interfax.ru/moscow/966535,Интерфакс,Площадь пожара составила 800 кв. м,2024-06-14
70,Медведев анонсирует учреждение комитета антине...,https://www.interfax.ru/world/966533,Интерфакс,Постоянный комитет антинеоколониального Движен...,2024-06-14
71,Запущенные хуситами ракеты попали по судну в А...,https://www.interfax.ru/world/966532,Интерфакс,На борту начался пожар,2024-06-14


In [12]:
rss = rss_raw.copy()
rss['content'] = rss['content'] + ' источник: ' +  rss['source']
rss = rss.drop('source', axis = 1)
rss

,title,link,content,formatted_dates
0,Have Good Health Insurance? Too Bad. You Could...,https://www.forbes.com/sites/reneehsia/2024/06...,Health insurance doesn't protect you from rece...,2024-06-14
1,Conor McGregor Is Out And UFC 303 Main Card Ge...,https://www.forbes.com/sites/brianmazique/2024...,Conor McGregor is officially off the UFC 303 c...,2024-06-14
2,The AI Revolution Comes for Farmers Growing a ...,https://www.bloomberg.com/news/articles/2024-0...,"<p>In its first test in Malawi, an AI app is h...",2024-06-14
3,AI Demand Helps Korean Chip Export Prices to R...,https://www.bloomberg.com/news/articles/2024-0...,The price of South Korea’s semiconductor expor...,2024-06-14
4,Microsoft Pulls Back Wide Release of Criticize...,https://www.bloomberg.com/news/articles/2024-0...,"Microsoft Corp., facing scrutiny over the secu...",2024-06-14
...,...,...,...,...
68,Силы ПВО отразили массовую атаку беспилотников...,https://www.interfax.ru/russia/966536,Массовая атака дронов отражена в Ростовской об...,2024-06-14
69,Кровля горящего здания обрушилась на севере Мо...,https://www.interfax.ru/moscow/966535,Площадь пожара составила 800 кв. м источник: И...,2024-06-14
70,Медведев анонсирует учреждение комитета антине...,https://www.interfax.ru/world/966533,Постоянный комитет антинеоколониального Движен...,2024-06-14
71,Запущенные хуситами ракеты попали по судну в А...,https://www.interfax.ru/world/966532,На борту начался пожар источник: Интерфакс,2024-06-14


In [13]:
import pandas as pd

# Предположим, что ваши датафреймы уже загружены как news_df и rss

# Переименуем столбцы в rss
rss_renamed = rss.rename(columns={
    'title': 'news_topic',
    'link': 'link',
    'content': 'news_text',
    'formatted_dates': 'news_date'
})

# Объединим два датафрейма
combined_df = pd.concat([news_df, rss_renamed], ignore_index=True)

combined_df

,link,news_date,news_text,news_topic
0,https://edtechmagazine.com/,,,
1,https://social.evraz.com/news/news/itogi-grant...,2024-06-14,,Итоги грантового конкурса социальных проектов ...
2,https://severstal.com/rus/media/,,,
3,https://www.forbes.com/sites/reneehsia/2024/06...,2024-06-14,Health insurance doesn't protect you from rece...,Have Good Health Insurance? Too Bad. You Could...
4,https://www.forbes.com/sites/brianmazique/2024...,2024-06-14,Conor McGregor is officially off the UFC 303 c...,Conor McGregor Is Out And UFC 303 Main Card Ge...
...,...,...,...,...
71,https://www.interfax.ru/russia/966536,2024-06-14,Массовая атака дронов отражена в Ростовской об...,Силы ПВО отразили массовую атаку беспилотников...
72,https://www.interfax.ru/moscow/966535,2024-06-14,Площадь пожара составила 800 кв. м источник: И...,Кровля горящего здания обрушилась на севере Мо...
73,https://www.interfax.ru/world/966533,2024-06-14,Постоянный комитет антинеоколониального Движен...,Медведев анонсирует учреждение комитета антине...
74,https://www.interfax.ru/world/966532,2024-06-14,На борту начался пожар источник: Интерфакс,Запущенные хуситами ракеты попали по судну в А...


In [14]:
# Получаем сегодняшнюю дату
today = datetime.now().date()

# Фильтруем по сегодняшней дате
combined_df['news_date'] = pd.to_datetime(combined_df['news_date'], format='%Y-%m-%d', errors='coerce')
filtered_df = combined_df[combined_df['news_date'].dt.date == today].reset_index(drop=True)
filtered_df

,link,news_date,news_text,news_topic
0,https://social.evraz.com/news/news/itogi-grant...,2024-06-14,,Итоги грантового конкурса социальных проектов ...
1,https://www.forbes.com/sites/reneehsia/2024/06...,2024-06-14,Health insurance doesn't protect you from rece...,Have Good Health Insurance? Too Bad. You Could...
2,https://www.forbes.com/sites/brianmazique/2024...,2024-06-14,Conor McGregor is officially off the UFC 303 c...,Conor McGregor Is Out And UFC 303 Main Card Ge...
3,https://www.bloomberg.com/news/articles/2024-0...,2024-06-14,"<p>In its first test in Malawi, an AI app is h...",The AI Revolution Comes for Farmers Growing a ...
4,https://www.bloomberg.com/news/articles/2024-0...,2024-06-14,The price of South Korea’s semiconductor expor...,AI Demand Helps Korean Chip Export Prices to R...
...,...,...,...,...
69,https://www.interfax.ru/russia/966536,2024-06-14,Массовая атака дронов отражена в Ростовской об...,Силы ПВО отразили массовую атаку беспилотников...
70,https://www.interfax.ru/moscow/966535,2024-06-14,Площадь пожара составила 800 кв. м источник: И...,Кровля горящего здания обрушилась на севере Мо...
71,https://www.interfax.ru/world/966533,2024-06-14,Постоянный комитет антинеоколониального Движен...,Медведев анонсирует учреждение комитета антине...
72,https://www.interfax.ru/world/966532,2024-06-14,На борту начался пожар источник: Интерфакс,Запущенные хуситами ракеты попали по судну в А...


In [15]:
ggl.update_gsheet_with_df(filtered_df,
                          'https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit#gid=0', 
                        4,
                         )